# get-images-by-staticmap

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image

import cv2
import os

# Set Input
#@markdown #Set Input
#@markdown Set a sequence of longitude/latitude pairs. (経度/緯度)
input_lon = 139.774444 #@param {type:"number"}
input_lat = 35.683611 #@param {type:"number"}

# Set path
#@markdown Set input csv & output images path.
use_csv_input = True #@param {type:"boolean"}
csv_path = "/content/drive/MyDrive/csv-dir" #@param {type:"string"}
csv_path += ".csv"

#@markdown #Set Output
#@markdown Set output images path.
output_dir = "/content/drive/MyDrive/output-dir" #@param {type:"string"}
output_name = "prefectural-capital-47" #@param {type:"string"}
output_path = output_dir
images_to_zip = True #@param {type:"boolean"}
if use_csv_input:
  output_path = output_dir + "/" + output_name

# Set info
#@markdown #Set Map Info
#@markdown 1. ベースマップ：標準地図（std-map）、ズームレベル1~18
#@markdown 2. ベースマップ：淡色地図（pale-map）、ズームレベル1~18
#@markdown 3. ベースマップ：写真（photo-map）、ズームレベル2~18
#@markdown 4. 標高・土地の凹凸：色別標高図（relief-map）、ズームレベル5~15
#@markdown 5. ベースマップ：English(english-map)、ズームレベル5~11
map_tile_url = 'https://cyberjapandata.gsi.go.jp/xyz/english/{z}/{x}/{y}.png' #@param ["https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png", "https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png", "https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg", "https://cyberjapandata.gsi.go.jp/xyz/relief/{z}/{x}/{y}.png", "https://cyberjapandata.gsi.go.jp/xyz/english/{z}/{x}/{y}.png"] {allow-input: true}
map_width = 1024 #@param {type:"number"}
map_height = 1024 #@param {type:"number"}
zoom_level = 11 #@param {type:"number"}
#@markdown Inflate the number of images by interpolating latitude and longitude.
number_of_position_interpolations =  0#@param {type:"number"}

In [ ]:
# @title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Create output dir
def make_outputfolder(dirname):
    path = Path(dirname)
    if path.exists() == False:
        Path(path).mkdir()

# make folder
if use_csv_input:
  make_outputfolder(output_path)

In [ ]:
# @title Install
# Install
!pip install staticmap

In [ ]:
# @title Run
from staticmap import StaticMap
import matplotlib.pyplot as plt

def save_map():
    map = StaticMap(map_width, map_height, url_template=map_tile_url)
    img = map.render(zoom=zoom_level, center=[input_lon, input_lat])

    plt.imshow(img)
    plt.show()

    img.save(output_path + "/" + output_name + ".png")

def save_map_by_csv():
  df = pd.read_csv(csv_path, encoding='utf-8')
  row_num = len(df)
  digit = len(str(row_num))

  map = StaticMap(map_width, map_height, url_template=map_tile_url)
  for index, row in df.iterrows():
    print(index+1, "/", row_num, ":", row[0], df.columns[1], row[1], df.columns[2], row[2])
    img = map.render(zoom=zoom_level, center=[row[2], row[1]])

    plt.imshow(img)
    plt.show()

    img.save('{}_{}.{}'.format(output_path + "/" + output_name, str(index).zfill(digit), "png"))

def save_map_by_csv_interpolations():
  df = pd.read_csv(csv_path, encoding='utf-8')

  df["faketime"] = list(range(0, len(df)*(number_of_position_interpolations+1), (number_of_position_interpolations+1)))
  df['faketime'] = pd.to_datetime(df['faketime'], unit='ms')
  df = df.set_index('faketime')
  df_interpolated = df.resample('1ms').interpolate()

  row_num = len(df_interpolated)
  digit = len(str(row_num))
  map = StaticMap(map_width, map_height, url_template=map_tile_url)

  tmp_index = 0
  for index, row in df_interpolated.iterrows():
    print(tmp_index, "/", row_num, ":", row[0], df_interpolated.columns[1], row[1], df_interpolated.columns[2], row[2])
    img = map.render(zoom=zoom_level, center=[row[2], row[1]])

    plt.imshow(img)
    plt.show()

    img.save('{}_{}.{}'.format(output_path + "/" + output_name, str(tmp_index).zfill(digit), "png"))
    tmp_index += 1

# Run
if not use_csv_input:
  save_map()
else:
  if number_of_position_interpolations > 0:
    save_map_by_csv_interpolations()
  else:
    save_map_by_csv()

In [ ]:
# Convert to zip
if images_to_zip and use_csv_input:
  dataset_name = output_name
  zip_name = dataset_name + ".zip"
  print(dataset_name, "->", zip_name)
  %cd $output_dir
  !zip $zip_name -r $dataset_name